# LLM Session Memory - Scope

Managing user input requires attention to access patterns and controling which applications can access which pieces of data. To help with this, the `SemanticSessionManager` class includes the `scope` parameter that controls the level of shared access.
We'll use the example of a travel agent application to illustrate different access levels.

Start by creating a simple wrapper around your LLM provider of choice. In this case we'll use OpenAI.

In [1]:
from typing import Dict, List
import os
from openai import OpenAI

class OpenAIClient():
    def __init__(self, api_key: str = None, model:str ='gpt-3.5-turbo'):
        api_key = api_key or os.getenv("OPENAI_API_KEY")
        self.client = OpenAI(api_key=api_key)
        self._model = model

    def converse(self, prompt: str, context: List[Dict], max_tokens: int = 250) -> str:
        context = self._remap(context)
        messages = context + [{"role": "user", "content": prompt}]
        response = self.client.chat.completions.create(
            model = self._model,
            messages = messages,
            max_tokens = max_tokens
        )
        return response.choices[0].message.content

    def _remap(self, context: List[Dict]) -> List[Dict]:
        ''' re-index the chat history to match the OpenAI API requirements '''
        new_context = []
        for statement in context:
            if statement["role"] == "_user":
                new_statement = {"role": "user", "content": statement["_content"]}
            elif statement["role"] == "_llm":
                new_statement = {"role": "assistant", "content": statement["_content"]}
            elif statement["role"] == "_preamble":
                new_statement = {"role": "system", "content": statement["_content"]}
            else:
                raise ValueError(f'Unknown chat role {statement["role"]}')
            new_context.append(new_statement)
        return new_context

We'll create 3 different session managers each with different levels of data access.
The narrowest scope is `session` where only prompts and responses passed to this manager are retrievable.
The next level is `user`, where all sessions of this specific user will be retrievable
The highesst level is `application` where all prompts and responses with the same application_id will be retrievable

In [2]:
from redisvl.extensions.session_manager import SemanticSessionManager
local_session = SemanticSessionManager(name='example', session_id='123', user_id='abc', scope='session')
local_session.set_preamble("You are a helpful travel agent, assisting people in planning popular vacations")
client = OpenAIClient()

In [3]:
def converse(prompt, session_mngr):
    context = session_mngr.fetch_recent()
    response = client.converse(prompt=prompt, context=context)
    session_mngr.store(prompt, response)
    print('USER: ', prompt)
    print('\nLLM: ', response)
    print('Conversation:')
    for ctx in context:
        print(ctx)

We can carry on several different conversations and inspect how data is accessible, or not accessible across different session instances.

In [4]:
converse("Can you help me plan a vacation to Scandinavia?", local_session)

USER:  Can you help me plan a vacation to Scandinavia?

LLM:  Of course! Scandinavia is a beautiful region known for its stunning landscapes, rich history, and vibrant culture. Here are some recommendations for your trip to Scandinavia:

1. **Destinations**: Consider visiting cities like Copenhagen in Denmark, Stockholm in Sweden, Oslo in Norway, and Helsinki in Finland. Each city has its own unique charm and attractions worth exploring.

2. **Activities**: Enjoy outdoor activities such as hiking in Norway's fjords, exploring the ancient Viking history in Sweden, and visiting historic landmarks like the Royal Palace in Copenhagen. Don't forget to experience the famous sauna culture in Finland!

3. **Transportation**: Scandinavia has an efficient transportation system, including trains, buses, and ferries. Consider getting a Scandinavian Rail Pass for easy travel between countries.

4. **Accommodation**: There are plenty of accommodation options ranging from luxury hotels to cozy cabins

In [5]:
converse("I plan to travel this Fall for 2 weeks and really enjoy beautiful hikes. I have a reasonable, but not extravagant budget prefer active vacations.", local_session)

USER:  I plan to travel this Fall for 2 weeks and really enjoy beautiful hikes. I have a reasonable, but not extravagant budget prefer active vacations.

LLM:  For your fall trip to Scandinavia focusing on beautiful hikes and active vacations, here's a suggested itinerary that combines outdoor adventures, stunning landscapes, and cultural experiences within a reasonable budget:

1. **Norway**:
   - **Destination**: Start your trip in Oslo, the capital of Norway, and explore the city's museums and parks. Then head to the famous fjords region, such as Geirangerfjord or Nærøyfjord, for breathtaking hikes surrounded by stunning scenery.
   - **Activities**: Hike the iconic Trolltunga or Preikestolen trails for panoramic views, and consider taking a scenic train ride on the Flåm Railway.
   - **Budget Accommodation**: Stay in budget-friendly hostels or guesthouses in the fjords region or opt for camping to immerse yourself in nature.

2. **Sweden**:
   - **Destination**: Travel to Stockholm

Unsurprisingly, the `local_session` mananger has access it it's own conversation history.
User level sessions with the same user id, in this case `abc` will also have access.
Let's create a new session with the same user, but different session id, and set its scope to `user`

In [6]:
user_session = SemanticSessionManager(name='example', session_id='456', user_id='abc', scope='user')
user_session.set_preamble("You are a helpful travel agent, assisting people in planning popular vacations")
context = user_session.fetch_recent()
for ctx in context:
    print(ctx)

17:57:33 redisvl.index.index INFO   Index already exists, not overwriting.
{'role': '_preamble', '_content': 'You are a helpful travel agent, assisting people in planning popular vacations'}
{'role': '_user', '_content': 'Can you help me plan a vacation to Scandinavia?'}
{'role': '_llm', '_content': 'Of course! Scandinavia is a beautiful region known for its stunning landscapes, rich history, and vibrant culture. Here are some recommendations for your trip to Scandinavia:\n\n1. **Destinations**: Consider visiting cities like Copenhagen in Denmark, Stockholm in Sweden, Oslo in Norway, and Helsinki in Finland. Each city has its own unique charm and attractions worth exploring.\n\n2. **Activities**: Enjoy outdoor activities such as hiking in Norway\'s fjords, exploring the ancient Viking history in Sweden, and visiting historic landmarks like the Royal Palace in Copenhagen. Don\'t forget to experience the famous sauna culture in Finland!\n\n3. **Transportation**: Scandinavia has an effici

We can carry on the conversation here, or switch topics.

In [7]:
converse("that sounds great for week 1, what about for week 2?", user_session)

USER:  that sounds great for week 1, what about for week 2?

LLM:  For the second week of your fall trip to Scandinavia, here are some suggestions for a continuation of your active vacation:

3. **Finland**:
   - **Destination**: Fly from Stockholm to Helsinki, the capital of Finland, and explore the city's vibrant art and design scene. Consider a day trip to the nearby Nuuksio National Park for hiking trails through forests and lakes.
   - **Activities**: Experience the Finnish sauna tradition and possibly go on a guided hiking tour in Lapland to explore the Arctic wilderness. Don't miss the chance to see the Northern Lights if you're lucky.
   - **Budget Accommodation**: Stay in cozy guesthouses or cottages in Lapland or opt for a wilderness cabin experience for a truly immersive outdoor stay.

4. **Denmark**:
   - **Destination**: Fly from Helsinki to Copenhagen, Denmark's capital known for its historic sites, colorful waterfront, and charming neighborhoods. Spend a couple of days e

The local session, with narrower scope, won't see this new conversation section.

In [8]:
for ctx in local_session.fetch_recent():
    print(ctx)

{'role': '_preamble', '_content': 'You are a helpful travel agent, assisting people in planning popular vacations'}
{'role': '_user', '_content': 'Can you help me plan a vacation to Scandinavia?'}
{'role': '_llm', '_content': 'Of course! Scandinavia is a beautiful region known for its stunning landscapes, rich history, and vibrant culture. Here are some recommendations for your trip to Scandinavia:\n\n1. **Destinations**: Consider visiting cities like Copenhagen in Denmark, Stockholm in Sweden, Oslo in Norway, and Helsinki in Finland. Each city has its own unique charm and attractions worth exploring.\n\n2. **Activities**: Enjoy outdoor activities such as hiking in Norway\'s fjords, exploring the ancient Viking history in Sweden, and visiting historic landmarks like the Royal Palace in Copenhagen. Don\'t forget to experience the famous sauna culture in Finland!\n\n3. **Transportation**: Scandinavia has an efficient transportation system, including trains, buses, and ferries. Consider g

Data is still compartmentalized to specific user_ids, so no information is available from different users either.

In [9]:
new_user_session = SemanticSessionManager(name='example', session_id='abc', user_id='def')
converse("I'm planning a trip to southeast Asia. Which country in that region has the best hiking trails?", new_user_session)

17:57:39 redisvl.index.index INFO   Index already exists, not overwriting.
USER:  I'm planning a trip to southeast Asia. Which country in that region has the best hiking trails?

LLM:  Southeast Asia is a diverse region with many great hiking destinations. One country that is known for its beautiful hiking trails is Vietnam. In Vietnam, you can explore the stunning landscapes of Sapa, with its terraced rice fields and lush green mountains. The trek to the summit of Fansipan, the highest peak in Indochina, is also a popular challenge for hikers.

Another great hiking destination in Southeast Asia is Thailand. The northern region of Thailand, particularly around Chiang Mai and Chiang Rai, offers a variety of hiking trails through lush jungles, past waterfalls, and to remote hill tribe villages.

Indonesia is another country with amazing hiking opportunities, especially on the islands of Bali, Java, and Sumatra. In Bali, you can hike to the top of Mount Batur for a sunrise view over the i

There are some situations where one would want to see or share data across users.

In [12]:

app_session = SemanticSessionManager(name='example', session_id='789', user_id='def', scope='application')
app_session.set_preamble("You are a data analist working for a travel agency, helping to track, monitor and report vacation trends.")
converse('based on you previous conversation history, what are the trending vacation places that people are asking you about?', app_session)



17:58:10 redisvl.index.index INFO   Index already exists, not overwriting.
USER:  based on you previous conversation history, what are the trending vacation places that people are asking you about?

LLM:  Based on previous conversations and data analysis, the trending vacation destinations that people are asking about include:

1. Japan - Many travelers are interested in exploring the rich culture, delicious cuisine, and unique attractions of Japan, including Tokyo, Kyoto, Osaka, and the scenic countryside.

2. Iceland - The stunning natural landscapes of Iceland, including waterfalls, glaciers, hot springs, and the Northern Lights, have been drawing a lot of interest from travelers seeking adventure and natural beauty.

3. National Parks in the United States - Visiting national parks such as Yellowstone, Grand Canyon, Yosemite, and Zion has been a popular choice for travelers looking to experience the great outdoors and explore the diverse landscapes of the U.S.

4. Italy - The histor

In [13]:
app_session.fetch_recent()

[{'role': '_preamble',
  '_content': 'You are a data analist working for a travel agency, helping to track, monitor and report vacation trends.'},
 {'role': '_user',
  '_content': "I'm planning a trip to southeast Asia. Which country in that region has the best hiking trails?"},
 {'role': '_llm',
  '_content': 'Southeast Asia is a diverse region with many great hiking destinations. One country that is known for its beautiful hiking trails is Vietnam. In Vietnam, you can explore the stunning landscapes of Sapa, with its terraced rice fields and lush green mountains. The trek to the summit of Fansipan, the highest peak in Indochina, is also a popular challenge for hikers.\n\nAnother great hiking destination in Southeast Asia is Thailand. The northern region of Thailand, particularly around Chiang Mai and Chiang Rai, offers a variety of hiking trails through lush jungles, past waterfalls, and to remote hill tribe villages.\n\nIndonesia is another country with amazing hiking opportunities,

In [14]:
local_session.delete()